In [1]:
from importlib.metadata import version

pkgs = ["optalgotools"]
for p in pkgs:
    print(f"{p} version: {version(p)}")

optalgotools version: 1.0.0


In [1]:
# puzzle state class
import math

class State:
    OPERATORS = ["left", "up", "right", "down"]

    # Checks to see if the state is solvable
    def is_solvable(self):
        inv = 0
        lst = self.flatten()
        for i in range(self.N * self.N):
            for j in range(i + 1, self.N * self.N):
                if lst[i] != 0 and lst[j] != 0 and lst[i] > lst[j]:
                    inv += 1
        return inv % 2 == 0

    # Generates an id, which is just a string representation of the grid
    def __init__(self, grid, parent=None):
        self.N = len(grid)
        self.grid = grid
        self.parent = parent
        self.id = "".join([str(i) for i in self.flatten()])
        
    def __hash__(self):
        return hash(self.id)
    
    def __eq__(self, other):
        try:
            return self.id == other.id
        except:
            return self.id == other

    def get_distance(self):
        return 1

    # 2D list to 1D list
    def flatten(self):
        return [i for k in self.grid for i in k]

    # 1D list to 2D list
    def to_grid(lst, N):
        return [lst[i : i + N] for i in range(0, len(lst), N)]

    # check if a state is complete
    def is_solved(self):
        flat = self.flatten()
        return flat == sorted(flat)  

    # Trace the parents until the beginning
    def path(self):
        node = self
        path = []
        while node:
            path.append(node)
            node = node.parent
        return path[::-1]

    # Swap positions of two cells based on direction, if possible
    def move(state, direction):
        N = state.N
        lst_state = state.flatten()
        i = lst_state.index(0)
        # Move 4 directions
        if direction == "up" and i - N >= 0:
            lst_state[i - N], lst_state[i] = lst_state[i], lst_state[i - N]
        elif direction == "down" and i + N < N * N:
            lst_state[i + N], lst_state[i] = lst_state[i], lst_state[i + N]
        elif direction == "left" and (i - 1) // N == i // N:
            lst_state[i - 1], lst_state[i] = lst_state[i], lst_state[i - 1]
        elif direction == "right" and (i + 1) // N == i // N:
            lst_state[i + 1], lst_state[i] = lst_state[i], lst_state[i + 1]
        else:
            return None
        return State.to_grid(lst_state, N)

    def expand(self):
        children = [State.move(self, d) for d in State.OPERATORS]
        children = list(filter(None, children))
        return [State(child, self) for child in children]



In [2]:
from optalgotools.algorithms.graph_search import DFS

init_state = [[1,4,2],
              [3,7,5],
              [6,0,8]]

goal_state = [[0,1,2],
              [3,4,5],
              [6,7,8]]


init_state = State(init_state)
goal_state = State(goal_state)


# Check if there's even a solution
if not init_state.is_solvable():
    print("This puzzle is not solvable.")
else:
    # The actual solution
    solution = DFS(init_state, goal_state)
    print(f"Process time: {solution.time} s")
    print(f"Space required: {solution.space} bytes")
    print(f"Explored states: {solution.explored}")
    print(f"Number of steps: {len(solution.result)}")


Process time: 0.015625 s
Space required: 624 bytes
Explored states: 29
Number of steps: 30
